# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [2]:
# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from collections import Counter

In [3]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
messages.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [5]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
all_df = messages.merge(categories, how='outer', on='id')
all_df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
all_df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [8]:
# create a dataframe of the 36 individual category columns
cat_df = categories.categories.str.split(';', expand=True)
cat_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [9]:
# select the first row of the categories dataframe
row = cat_df.iloc[0,:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda s: s.split('-')[0]).tolist()
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [10]:
# rename the columns of `categories`
cat_df.columns = category_colnames
cat_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [11]:
for column in cat_df:
    # set each value to be the last character of the string
    cat_df[column] = cat_df[column].apply(lambda s: s[-1])
    
    # convert column from string to numeric
    cat_df[column] = pd.to_numeric(cat_df[column])
cat_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [12]:
# drop the original categories column from `df`
all_df.drop('categories', axis=1, inplace=True)

all_df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [13]:
# concatenate the original dataframe with the new `categories` dataframe
all_df = pd.concat([all_df, cat_df], axis=1)
all_df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [14]:
# check number of duplicates
all_df.duplicated(subset=None, keep='first').sum()

41

In [15]:
# drop duplicates
all_df.drop_duplicates(subset=None, keep='first', inplace=True)

In [16]:
# check number of duplicates
all_df.duplicated(subset=None, keep='first').sum()

0

In [17]:
# check what is wrong with 2s:
all_df.related.unique()

array([ 1.,  0.,  2., nan])

In [18]:
Counter(all_df.related.dropna().values)

Counter({0.0: 6101, 1.0: 19914, 2.0: 192})

As we see, there are 192 (< 1%) of 2s in all_df dataset. Try to figure out what is wrong with them:

In [19]:
all_df[all_df.related == 2.0].mean()

id                        11500.390625
related                       2.000000
request                       0.000000
offer                         0.000000
aid_related                   0.000000
medical_help                  0.000000
medical_products              0.000000
search_and_rescue             0.000000
security                      0.000000
military                      0.000000
child_alone                   0.000000
water                         0.000000
food                          0.000000
shelter                       0.000000
clothing                      0.000000
money                         0.000000
missing_people                0.000000
refugees                      0.000000
death                         0.000000
other_aid                     0.000000
infrastructure_related        0.000000
transport                     0.000000
buildings                     0.000000
electricity                   0.000000
tools                         0.000000
hospitals                

What about 0s and 1s?

In [20]:
all_df[all_df.related == 0].mean()

id                        13759.847074
related                       0.000000
request                       0.000000
offer                         0.000000
aid_related                   0.000000
medical_help                  0.000000
medical_products              0.000000
search_and_rescue             0.000000
security                      0.000000
military                      0.000000
child_alone                   0.000000
water                         0.000000
food                          0.000000
shelter                       0.000000
clothing                      0.000000
money                         0.000000
missing_people                0.000000
refugees                      0.000000
death                         0.000000
other_aid                     0.000000
infrastructure_related        0.000000
transport                     0.000000
buildings                     0.000000
electricity                   0.000000
tools                         0.000000
hospitals                

In [21]:
all_df[all_df.related == 1].mean()

id                        15601.455007
related                       1.000000
request                       0.224967
offer                         0.005976
aid_related                   0.546249
medical_help                  0.104801
medical_products              0.065984
search_and_rescue             0.036356
security                      0.023652
military                      0.043186
child_alone                   0.000000
water                         0.084061
food                          0.147133
shelter                       0.116451
clothing                      0.020388
money                         0.030330
missing_people                0.015015
refugees                      0.043989
death                         0.060058
other_aid                     0.173145
infrastructure_related        0.085618
transport                     0.060410
buildings                     0.067038
electricity                   0.026815
tools                         0.007984
hospitals                

As we see now, 2s are very similar to 0s. Can we make any distinctions between these two categories?

In [22]:
text_cols = ['id', 'message', 'original', 'genre']
all_df[all_df.related == 2.0][text_cols].groupby('genre').agg(len)['id']

genre
direct    129
news       18
social     45
Name: id, dtype: int64

In [23]:
all_df[all_df.related == 0.0][text_cols].groupby('genre').agg(len)['id']

genre
direct    3433
news      2353
social     315
Name: id, dtype: int64

In [24]:
all_df[all_df.related == 1.0][text_cols].groupby('genre').agg(len)['id']

genre
direct     7263
news      10607
social     2044
Name: id, dtype: int64

As we see, the 2s not only show messages that are not positively classified in any of categories (except `related`) but also different from 0s. Because 2s are very subdominant, it is appropriate to discard them from future analysis:

In [25]:
all_df = all_df[all_df.related < 2.0]
all_df.shape

(26015, 40)

In [26]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26015 entries, 0 to 26247
Data columns (total 40 columns):
id                        26015 non-null int64
message                   26015 non-null object
original                  10094 non-null object
genre                     26015 non-null object
related                   26015 non-null float64
request                   26015 non-null float64
offer                     26015 non-null float64
aid_related               26015 non-null float64
medical_help              26015 non-null float64
medical_products          26015 non-null float64
search_and_rescue         26015 non-null float64
security                  26015 non-null float64
military                  26015 non-null float64
child_alone               26015 non-null float64
water                     26015 non-null float64
food                      26015 non-null float64
shelter                   26015 non-null float64
clothing                  26015 non-null float64
money         

In [27]:
all_df.select_dtypes(include=['float64']).sum(axis=0).sort_values()

child_alone                   0.0
offer                       119.0
shops                       120.0
tools                       159.0
fire                        282.0
hospitals                   283.0
missing_people              299.0
aid_centers                 309.0
clothing                    406.0
security                    471.0
cold                        530.0
electricity                 534.0
money                       604.0
search_and_rescue           724.0
military                    860.0
refugees                    876.0
other_infrastructure       1151.0
death                      1196.0
transport                  1203.0
medical_products           1314.0
buildings                  1335.0
other_weather              1376.0
water                      1674.0
infrastructure_related     1705.0
medical_help               2087.0
floods                     2158.0
shelter                    2319.0
storm                      2448.0
earthquake                 2453.0
food          

As we see, it makes sense to exclude the `child_alone` feature as it does not contain positive-labeled instances
(which will result in zero `F-score` for this particular column). However, because the project rubric relies on prediction of 36 columns, we will keep this column using an appropriate metric (mean `F-score` among the target columns) instead.

In [28]:
#all_df.drop(['child_alone'], axis=1, inplace=True)

Other `float64` features have to be converted into `bool`:

In [29]:
for col in all_df.select_dtypes(include=['float64']):
    all_df[col] = all_df[col].astype(bool)

In [30]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26015 entries, 0 to 26247
Data columns (total 40 columns):
id                        26015 non-null int64
message                   26015 non-null object
original                  10094 non-null object
genre                     26015 non-null object
related                   26015 non-null bool
request                   26015 non-null bool
offer                     26015 non-null bool
aid_related               26015 non-null bool
medical_help              26015 non-null bool
medical_products          26015 non-null bool
search_and_rescue         26015 non-null bool
security                  26015 non-null bool
military                  26015 non-null bool
child_alone               26015 non-null bool
water                     26015 non-null bool
food                      26015 non-null bool
shelter                   26015 non-null bool
clothing                  26015 non-null bool
money                     26015 non-null bool
missing_pe

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [31]:
engine = create_engine('sqlite:///DisasterMessagesDatabase.db')
all_df.to_sql('DisasterMessagesDatabase', engine, index=False, if_exists='replace')

In [32]:
!ls -lotrh *db

-rw-rw-r-- 1 dima806 6.2M Nov  4 19:33 InsertDatabaseName.db
-rw-rw-r-- 1 dima806 6.5M Nov  4 19:34 DisasterResponse.db
-rw-rw-r-- 1 dima806 6.2M Nov 16 16:35 DisasterMessagesDatabase.db


In [34]:
all_df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [38]:
all_df.shape

(26015, 40)

In [39]:
engine = create_engine('sqlite:///DisasterMessagesDatabase.db')
df = pd.read_sql_table('DisasterMessagesDatabase', engine)
df.shape

(26015, 40)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [35]:
%%writefile process_data.py

# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

def make_etl(messages_str, categories_str, database_str):
    '''
    Extract-Transform-Load procedure for disaster messages and message categories
    
    Args:
        messages_str (string): a path to messages.csv file
        categories_str (string): a path to categories.csv file
        database_str (string): a path to final cleaned database

    Returns:
        None    
    
    '''

    # load messages dataset
    messages = pd.read_csv(messages_str)
    print('>>> messages shape: ', messages.shape)

    # load categories dataset
    categories = pd.read_csv(categories_str)
    print('>>> categories shape: ', categories.shape)

    # merge datasets
    all_df = messages.merge(categories, how='outer', on='id')
    print('>>> all_df shape: ', all_df.shape)

    # create a dataframe of the 36 individual category columns
    cat_df = categories.categories.str.split(';', expand=True)
    print('>>> cat_df shape: ', cat_df.shape)

    # select the first row of the categories dataframe
    row = cat_df.iloc[0,:]

    # use this row to extract a list of new column names for categories.
    # one way is to apply a lambda function that takes everything 
    # up to the second to last character of each string with slicing
    category_colnames = row.apply(lambda s: s.split('-')[0]).tolist()
    print('>>> category colnames: ', category_colnames)

    # rename the columns of `categories`
    cat_df.columns = category_colnames

    # convert cat_df values to 0 and 1
    for column in cat_df:
        # set each value to be the last character of the string
        cat_df[column] = cat_df[column].apply(lambda s: s[-1])
        # convert column from string to numeric
        cat_df[column] = pd.to_numeric(cat_df[column])

    # drop the original categories column from `df`
    all_df.drop('categories', axis=1, inplace=True)

    # concatenate the original dataframe with the new `categories` dataframe
    all_df = pd.concat([all_df, cat_df], axis=1)
    
    # sort out 2s from `related` column
    all_df = all_df[all_df.related < 2.0]
        
    # convert all float64 columns to bool
    for col in all_df.select_dtypes(include=['float64']):
        all_df[col] = all_df[col].astype(bool)
    
    # check number of duplicates
    print('>>> number of duplicates before dropping: ', all_df.duplicated(subset=None, keep='first').sum())

    # drop duplicates
    all_df.drop_duplicates(subset=None, keep='first', inplace=True)

    # check number of duplicates
    print('>>> number of duplicates after dropping: ', all_df.duplicated(subset=None, keep='first').sum())

    # print the final shape
    print('>>> all_df shape: ', all_df.shape)

    engine = create_engine('sqlite:///'+database_str)
    all_df.to_sql('DisasterMessagesDatabase', engine, index=False, if_exists='replace')
    
    print('>>> DONE')
    
    return all_df

if __name__ == '__main__':
    messages_str, categories_str, database_str = sys.argv[1:]
    print(messages_str, categories_str, database_str)
    make_etl(messages_str, categories_str, database_str)

Overwriting process_data.py


In [36]:
!ls -lotr process_data.py

-rw-r--r-- 1 dima806 3171 Nov 16 16:36 process_data.py
